# Find punishment txs

In [1]:
from utils import *
from api_calls import *
import pandas as pd

In [5]:
settlement_txs_dict = read_json(level1_folder + 'settlement_txs_dict.json')

## select settlement txs with two outputs

In [7]:
possible_two_output_unilateral_settlement_txs_hashes = []
for tx in settlement_txs_dict.values():
    if 'outputs' in tx and len(tx['outputs']) == 2:
        possible_two_output_unilateral_settlement_txs_hashes.append(tx['tx_hash'])
    elif 'vout' in tx and len(tx['vout']) == 2:
        possible_two_output_unilateral_settlement_txs_hashes.append(tx['txid'])

In [9]:
len(settlement_txs_dict)

61189

In [8]:
len(possible_two_output_unilateral_settlement_txs_hashes)

25686

### get tx details from blockstream

In [10]:
possible_two_output_unilateral_settlement_txs = read_json(level1_folder + 'possible_two_output_unilateral_settlement_txs.json')

In [11]:
len(possible_two_output_unilateral_settlement_txs)

11935

In [12]:
type(possible_two_output_unilateral_settlement_txs)

dict

In [13]:
for hsh in possible_two_output_unilateral_settlement_txs_hashes:
    if hsh not in possible_two_output_unilateral_settlement_txs:
        possible_two_output_unilateral_settlement_txs[hsh] = None

In [14]:
len(possible_two_output_unilateral_settlement_txs)

25686

In [16]:
for i, hsh in enumerate(possible_two_output_unilateral_settlement_txs_hashes):
#     print(i, end='\r')
    if not possible_two_output_unilateral_settlement_txs[hsh]:
        if hsh in settlement_txs_dict and 'txid' in settlement_txs_dict[hsh]:
            possible_two_output_unilateral_settlement_txs[hsh] = settlement_txs_dict[hsh]
        else:
            possible_two_output_unilateral_settlement_txs[hsh] = get_blockstream_tx(hsh)

# select unilateral close

In [17]:
bilateral_seq = 0
unilateral_seq = 0
for t in possible_two_output_unilateral_settlement_txs.values():
    seq = t['vin'][0]['sequence']
    if seq != 4294967295:
        unilateral_seq += 1
    else:
        bilateral_seq += 1
print('bilateral_seq', bilateral_seq)
print('unilateral_seq', unilateral_seq)


bilateral_seq 16072
unilateral_seq 9614


In [18]:
double_p2wsh = 0
p2wsh_outputs = set() # tx_hash:out_index
n_p2wsh = 0
unilateral_settlement_txs_with_p2wsh_in_outputs = []
for t in possible_two_output_unilateral_settlement_txs.values():
    seq = t['vin'][0]['sequence']
    if seq != 4294967295:
        # unilateral
        n_p2wsh_in_tx = 0
        for i, vout in enumerate(t['vout']):
            if vout['scriptpubkey_type'] == 'v0_p2wsh':
                p2wsh_outputs.add(t['txid'] + ':' + str(i))
                n_p2wsh_in_tx += 1
        if n_p2wsh_in_tx:
            unilateral_settlement_txs_with_p2wsh_in_outputs.append(t['txid'])
        if n_p2wsh_in_tx == 2:
            double_p2wsh += 1
#             print(t['txid'])
        n_p2wsh += n_p2wsh_in_tx
print('Out of', len(possible_two_output_unilateral_settlement_txs_hashes), 'two-output settlement txs, we have', 
      unilateral_seq, 'unilateral closes and', double_p2wsh, 'of them have p2wsh in both outputs')

Out of 25686 two-output settlement txs, we have 9614 unilateral closes and 85 of them have p2wsh in both outputs


In [19]:
len(p2wsh_outputs)

9699

In [20]:
len(unilateral_settlement_txs_with_p2wsh_in_outputs)

9614

## get spenders of p2wsh from blockchain explorer

### save entire page of unilateral txs

## create dict where key: unilateral close, value: spender hashes if output is p2wsh

In [21]:
unilateral_settlement_txs_with_p2wsh_in_outputs = []
for t in possible_two_output_unilateral_settlement_txs.values():
    seq = t['vin'][0]['sequence']
    if seq != 4294967295:
        # unilateral
        n_p2wsh_in_tx = 0
        for i, vout in enumerate(t['vout']):
            if vout['scriptpubkey_type'] == 'v0_p2wsh':
                p2wsh_outputs.add(t['txid'] + ':' + str(i))
                n_p2wsh_in_tx += 1
        if n_p2wsh_in_tx:
            unilateral_settlement_txs_with_p2wsh_in_outputs.append(t['txid'])
        if n_p2wsh_in_tx == 2:
            double_p2wsh += 1
        n_p2wsh += n_p2wsh_in_tx

In [24]:
hash_page = dict()
for tx_hash in unilateral_settlement_txs_with_p2wsh_in_outputs:
    hash_page[tx_hash] = None

In [41]:
len(hash_page)

9614

In [44]:
hash_spenders = dict()
for i in [1, 2, 3, 4, 5]:
    partial_hash_spenders = read_json(level1_folder + str(i) + '.json')
    for hsh, spenders in partial_hash_spenders.items():
        hash_spenders[hsh] = spenders

In [45]:
len(hash_spenders)

9614

In [46]:
write_json(list(hash_spenders), level1_folder + 'hash_spenders.json')

In [47]:
unilateral_settlement_txs_with_p2wsh_in_outputs_spenders = dict()
for tx_hash in unilateral_settlement_txs_with_p2wsh_in_outputs:
    unilateral_settlement_txs_with_p2wsh_in_outputs_spenders[tx_hash] = hash_spenders[tx_hash]

In [48]:
spender_details = read_json(level1_folder + 'spender_details.json')

In [49]:
len(spender_details)

3901

In [50]:
for stx in unilateral_settlement_txs_with_p2wsh_in_outputs_spenders:
    spenders = unilateral_settlement_txs_with_p2wsh_in_outputs_spenders[stx]
    for i, spender in enumerate(spenders): # two spender txs at most
        stx_out_index = stx + ':' + str(i)
        if stx_out_index in p2wsh_outputs and spender not in spender_details:
            spender_details[spender] = None


In [52]:
len(spender_details)

7641

In [55]:
for i, tx in enumerate(spender_details):
    print(i, end='\r')
    if not spender_details[tx]:
        details = get_blockstream_tx(tx)
        spender_details[tx] = details

3467

In [56]:
write_json(spender_details, level1_folder + 'spender_details.json')

In [64]:
# for each spender I need not only the tx hash,
# but also the address and the amount to be more secure
spender_address_btc = set()  # spender tx hash + address + btc
for stx in unilateral_settlement_txs_with_p2wsh_in_outputs_spenders:
    spenders = unilateral_settlement_txs_with_p2wsh_in_outputs_spenders[stx]
    for i, spender in enumerate(spenders):  # two spender txs at most
        stx_out_index = stx + ':' + str(i)
        if stx_out_index in p2wsh_outputs:
            if 'outputs' in settlement_txs_dict[stx]:
                address = settlement_txs_dict[stx]['outputs'][i]['address']  # the output address of the settlement tx
                btc = settlement_txs_dict[stx]['outputs'][i]['value']['value']
            elif 'vout' in settlement_txs_dict[stx]:
                address = settlement_txs_dict[stx]['vout'][i]['scriptpubkey_address']  # the output address of the settlement tx
                btc = settlement_txs_dict[stx]['vout'][i]['value']
            key = spender + ':' + address + ':' + str(btc)
            spender_address_btc.add(key)


In [65]:
write_json(list(spender_address_btc), level1_folder + 'spender_address_btc.json')

In [69]:
output_file = results_folder + 'settlement_txs_with_punishment.json'

In [ ]:
# spender_address_btc = read_json(input_file1)
# spender_details = read_json(input_file2)
# possible_two_output_unilateral_settlement_txs = read_json(input_file3)


In [66]:
non_collaborative_settlement_address_btc = dict()
non_standard_witnesses = []
n_punishment_txs = 0
non_standard_witness_len = dict()
for sab in spender_address_btc:
    spender, address, btc = sab.split(':')
    details = spender_details[spender]  # cannot use spender_address_btc_details cause it has more keys
    for vin in details['vin']:
        if vin['prevout']['scriptpubkey_address'] == address and vin['prevout']['value'] == int(btc):
            len_witness = len(vin['witness'])
            if len_witness != 3:
                if len_witness not in non_standard_witness_len:
                    non_standard_witness_len[len_witness] = 0
                non_standard_witness_len[len_witness] += 1
                # print(details['txid'], 'non standard witness')
                non_standard_witnesses.append(vin['witness'])
#                 print(len(vin['witness']), end=' ')
            else:
                x = vin['witness'][1]
                if x:
                    # print(spender, x)
                    n_punishment_txs += 1
                    key = address + ':' + btc
                    if key not in non_collaborative_settlement_address_btc:
                        non_collaborative_settlement_address_btc[key] = 0
                    non_collaborative_settlement_address_btc[key] += 1


In [67]:
settlement_txs_with_punishment = set()
# for each settlement tx
for tx in possible_two_output_unilateral_settlement_txs:
    # if one output has a p2wsh and the address and the value are in non_collaborative_settlement_address_btc
        # add settlement tx to unusable
    for vout in possible_two_output_unilateral_settlement_txs[tx]['vout']:
        if vout['scriptpubkey_type'] == 'v0_p2wsh':
            key = vout['scriptpubkey_address'] + ':' + str(vout['value'])
            if key in non_collaborative_settlement_address_btc:
                settlement_txs_with_punishment.add(tx)

In [68]:
len(settlement_txs_with_punishment)

12

In [71]:
output_file

'../data/joined/results/settlement_txs_with_punishment.json'

In [ ]:
res

In [70]:
write_json(list(settlement_txs_with_punishment), output_file)